In [1]:
%load_ext autoreload
%autoreload 2
# imports
import time

# important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scipy as sp

from tqdm.auto import tqdm
import itertools

# for better printing of variables
from IPython.display import display

# custom imports
from concperf import single_model, general_model
from concperf import utility

In [2]:
# update configuration dictionary for each instance count
def update_config(config):
    config['arrival_rate_server'] = config['arrival_rate_total'] / config['instance_count']
    config['base_service_time'] = config['base_service_time_ms'] / 1000
    config['stable_conc_avg_count'] = 10 * config['instance_count']

model_config = {
    # 'instance_count' should be added for each state
    'max_conc': 100,
    'arrival_rate_total': 10,
    'alpha': 0.11,
    'base_service_time_ms': 1154,
    'max_container_count': 25,
    'target_conc': 2*0.7, # assumes target utilization
    'max_scale_up_rate': 1000, # from N to 1000*N at most
    'max_scale_down_rate': 2, # from N to N/2 at most
    # 'stable_conc_avg_count': 60, # number of times monitored concurrency will be averaged in stable mode
    'autoscaling_interval': 2, # amount of time between autoscaling evaluations
    'provision_rate_base': 1,
    'deprovision_rate_base': 2,
}

# test
# model_config = {
#     # 'instance_count' should be added for each state
#     'max_conc': 100,
#     'arrival_rate_total': 15,
#     'alpha': 1,
#     'base_service_time_ms': 400,
#     'max_container_count': 25,
#     'target_conc': 16*0.7, # assumes target utilization
#     'max_scale_up_rate': 1000, # from N to 1000*N at most
#     'max_scale_down_rate': 2, # from N to N/2 at most
#     # 'stable_conc_avg_count': 60, # number of times monitored concurrency will be averaged in stable mode
#     'autoscaling_interval': 2, # amount of time between autoscaling evaluations
#     'provision_rate_base': 1,
#     'deprovision_rate_base': 2,
# }

In [3]:
# sample execution
# res = general_model.solve_general_model(model_config, update_config)
# res_params = general_model.calculate_general_params(res, model_config)
# res.update(res_params)
# res.keys()

# Arrival Rate Plots

In [4]:
arrival_rates = list(np.linspace(1, 20, 10))

def parse_arrival_rate(arrival_rate):
    config = {**model_config}
    config.update({
        'arrival_rate_total': arrival_rate,
    })
    res = general_model.solve_general_model(config, update_config)
    res_params = general_model.calculate_general_params(res, config)
    res.update(res_params)
    res.update(config)
    return res

start_time = time.time()
total_results = list(map(parse_arrival_rate, tqdm(arrival_rates, position=0, leave=True)))
elapsed_time = time.time() - start_time
print(f"\nnew order calculation took {elapsed_time} seconds for {len(arrival_rates)} arrival rates ({elapsed_time/len(arrival_rates):4.2f} per job)")

100%|██████████| 10/10 [05:53<00:00, 35.38s/it]
new order calculation took 353.8593850135803 seconds for 10 arrival rates (35.39 per job)



In [7]:
extract_keys = [
    'arrival_rate_total', 
    'ready_avg', 
    'ordered_avg', 
    'req_count_avg', 
    'resp_time_avg', 
]

extract_vals = []
for r in total_results:
    extract_vals.append({k:r[k] for k in extract_keys})

pd.DataFrame(data=extract_vals)

,arrival_rate_total,ready_avg,ordered_avg,req_count_avg,resp_time_avg
0,1.000000,1.298907,1.326205,1.111127,1250.475252
1,3.111111,3.366763,3.402555,1.255345,1260.270016
2,5.222222,5.370668,5.415210,1.308557,1264.105955
3,7.333333,7.373173,7.426605,1.335912,1266.357849
4,9.444444,9.356835,9.416909,1.354622,1267.946534
5,11.555556,11.375936,11.443492,1.362570,1268.621667
6,13.666667,13.373917,13.446792,1.371017,1269.412314
7,15.777778,15.379620,15.459583,1.377570,1270.103421
8,17.888889,17.364914,17.449889,1.384200,1270.770915
9,20.000000,19.361245,19.452688,1.389618,1271.393905
